#### Initial Setup

In [ ]:
#imported libraries
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
import langdetect
import datetime
%matplotlib inline  
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import Imputer

import findspark; findspark.init()
import pyspark
import os
from pyspark.sql.types import *
import pyspark.sql.functions as func

os.environ["PYSPARK_SUBMIT_ARGS"] = (
  "--packages com.databricks:spark-csv_2.11:1.4.0 pyspark-shell"
)

try:
    sc = pyspark.SparkContext()
except Exception as e:
    print "SparkContext exists... Continuing on."
    
sqlCtx = pyspark.sql.SQLContext(sc)

#### Load files

In [ ]:
downloads = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_app_downloads.csv').drop('')
ratings = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_app_rating.csv').drop('')
usages = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_usage.csv').drop('')
revenues = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_revenue.csv').drop('')
output = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_final_downloads.csv').drop('')
prev_downloads = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_cumulative_downloads_2015-02.csv').drop('')  
release_date = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_release_date.csv').drop('')
    
reviews = pd.read_csv('train_app_review.csv')
reviews_schema = StructType([
    StructField("id",IntegerType(),True),
    StructField("name",StringType(),True),
    StructField("country",StringType(),True),
    StructField("rating",IntegerType(),True),
    StructField("date",StringType(),True),
    StructField("title",StringType(),True),
    StructField("version",StringType(),True),
    StructField("text",StringType(),True),
    StructField("reviewer",StringType(),True)
])
reviews = sqlCtx.createDataFrame(reviews,reviews_schema)

#### Generate Predictors

In [ ]:
old_dateRange = pd.date_range('2015-03-01', periods=56).format(formatter=lambda x: x.strftime('%Y-%m-%d'))
dateRange = pd.date_range('2015-03-01', periods=56).format(formatter=lambda x: x.strftime('%Y_%m_%d'))
for d in range(56):
    revenues = revenues.withColumnRenamed(old_dateRange[d],dateRange[d])
    usages = usages.withColumnRenamed(old_dateRange[d],dateRange[d])
    downloads = downloads.withColumnRenamed(old_dateRange[d],dateRange[d])

In [ ]:
#Initialization
predictors = downloads['id','name','category','device']

In [ ]:
# Generate the weekly downloads
sqlCtx.registerDataFrameAsTable(downloads, "downloads")
predictors = sqlCtx.sql("SELECT id, name, category, device , "+\
           "+".join(dateRange[0:7])+" AS week_1 ,"+\
           "+".join(dateRange[7:14])+" AS week_2 ,"+\
           "+".join(dateRange[14:21])+" AS week_3 ,"+\
           "+".join(dateRange[21:28])+" AS week_4 ,"+\
           "+".join(dateRange[28:35])+" AS week_5 ,"+\
           "+".join(dateRange[35:42])+" AS week_6 ,"+\
           "+".join(dateRange[42:49])+" AS week_7 ,"+\
           "+".join(dateRange[49:56])+" AS week_8 ,"+\
           "+".join(dateRange)+" AS download_sum \
           from downloads")

In [ ]:
# Make coefficients for download regression
def get_coefficients(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    return  float(np.polyfit(range(56),np.cumsum(args[1:]),args[0])[0])
#Generate the step max and min 
def get_maxStep(maximum,*args):
    args=list(args)
    if (np.count_nonzero(args) == 0):
        return 0
    m = 0
    for d in range(1,56):
        if (args[d]!=0 and args[d-1]!=0):
            c = (args[d]-args[d-1])
            if (maximum and m < c):
                m = c
            if ( not maximum and m > c):
                m = c
    return m
#generate standard deviation
def get_std(*args):
    return float(np.std(list(args)))
#generate the number of missing values (imputation must be done after this)
def get_nbMissing(*args):
    return list(args).count(-1)

sqlCtx.registerFunction("get_nbMissing", get_nbMissing,returnType=IntegerType())
sqlCtx.registerFunction("get_std", get_std,returnType=FloatType())
sqlCtx.registerFunction("get_maxStep", get_maxStep,returnType=IntegerType())
sqlCtx.registerFunction("get_coefficients", get_coefficients,returnType=FloatType())
temp_downloads = sqlCtx.sql("SELECT id,name,category, device \
,get_coefficients(0,"+",".join(dateRange)+") AS coef_0 \
,get_coefficients(1,"+",".join(dateRange)+") AS coef_1 \
,get_coefficients(2,"+",".join(dateRange)+") AS coef_2 \
,get_coefficients(3,"+",".join(dateRange)+") AS coef_3 \
,get_maxStep(True,"+",".join(dateRange)+") AS max_step \
,get_maxStep(False,"+",".join(dateRange)+") AS min_step \
,get_std("+",".join(dateRange)+") AS downloads_std \
,get_nbMissing("+",".join(dateRange)+") AS nb_missing \
 FROM downloads")

predictors = predictors.join(temp_downloads,["id","name","category","device"])

In [ ]:
# previous downloads addition
predictors = predictors.join(prev_downloads,["id","device"])

In [ ]:
# Days since release generation
def get_days(date):
    return (datetime.datetime.strptime('03/01/2015', '%m/%d/%Y').date() \
            - datetime.datetime.strptime(date, '%Y-%m-%d').date()).days

sqlCtx.registerDataFrameAsTable(release_date, "release_date")
sqlCtx.registerFunction("get_days", get_days,returnType=IntegerType())
temp_date = sqlCtx.sql("SELECT id,name \
, get_days(release_date) AS days_since_release \
 FROM release_date")

predictors = predictors.join(temp_date,["id","name"],"left")

In [ ]:
#ratings generation
sqlCtx.registerDataFrameAsTable(ratings, "ratings")
temp_ratings = sqlCtx.sql("SELECT id,name,category \
, start1/(start1+star2+star3+star4+star5) AS star1 \
, star2/(start1+star2+star3+star4+star5) AS star2 \
, star3/(start1+star2+star3+star4+star5) AS star3 \
, star4/(start1+star2+star3+star4+star5) AS star4 \
, star5/(start1+star2+star3+star4+star5) AS star5 \
, (start1+star2+star3+star4+star5) AS num_ratings \
 FROM ratings")

predictors = predictors.join(temp_ratings,["id","name","category"],"left")

In [ ]:
# Categories
list_categories = [ x.category.replace(" ","_") for x in sqlCtx.sql("SELECT category \
 FROM downloads\
 group by category \
 ").collect()]
for cat in list_categories:
    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    predictors=sqlCtx.sql('''SELECT *, CASE WHEN (category = "'''+cat+'''") THEN 1 ELSE 0 END AS '''+cat+''' FROM predictors''')


In [ ]:
# Device
sqlCtx.registerDataFrameAsTable(predictors, "predictors")
predictors=sqlCtx.sql('''SELECT *, CASE WHEN (device = "iphone") THEN 1 ELSE 0 END AS iphone FROM predictors''')
sqlCtx.registerDataFrameAsTable(predictors, "predictors")
predictors=sqlCtx.sql('''SELECT *, CASE WHEN (device = "ipad") THEN 1 ELSE 0 END AS ipad FROM predictors''')

In [ ]:
def get_language(x):
    try:
        detected = langdetect.detect_langs(x.decode('utf8','ignore'))[0]
        if detected.prob < 0.7:
            return "other"
        else :
            return detected.lang
    except:
        return "other"
sqlCtx.registerFunction("get_language", get_language,returnType=StringType())

In [ ]:
#Language of the title
lang = ['ja','zh-cn','ko','en']
for l in lang:
    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    predictors=sqlCtx.sql('''SELECT *, CASE WHEN (get_language(name) = "'''+l+'''") THEN 1 \
    ELSE 0 END AS '''+l.replace("-","_")+''' FROM predictors''')

In [ ]:
# Reviews
sqlCtx.registerDataFrameAsTable(reviews, "reviews")
temp_reviews=sqlCtx.sql('''SELECT *, get_language(text) AS text_language FROM reviews''')
temp_reviews.head(1)

#language
sqlCtx.registerDataFrameAsTable(temp_reviews, "temp_reviews")
list_languages = [ x.text_language.replace(" ","_") for x in sqlCtx.sql("SELECT text_language \
 FROM temp_reviews\
 group by text_language \
 ").collect()]
print list_languages
#escape is used in case some asshole used - or [space] anywhere
def escape(text):
    return text.replace(" ","_").replace("-","_")
sqlCtx.registerFunction("escape", escape,returnType=StringType())
#sentiment
def get_sentiment(text,title):
    return 0.5
sqlCtx.registerFunction("get_sentiment", get_sentiment,returnType=FloatType())
# number of reviews
def get_recentReviews(date):
    return int((datetime.datetime.strptime('03/01/2015', '%m/%d/%Y').date() \
            - datetime.datetime.strptime(date, '%Y-%m-%d').date()).days >=0)
sqlCtx.registerFunction("get_recentReviews", get_recentReviews,returnType=IntegerType())

#languages expanded
for l in list_languages:
    sqlCtx.registerDataFrameAsTable(temp_reviews, "temp_reviews")
    temp_reviews=sqlCtx.sql('''SELECT *, CASE WHEN (text_language = "'''+l+'''") THEN 1 \
    ELSE 0 END AS '''+l.replace("-","_")+''' FROM temp_reviews''')
 
#sentiment, 1 and recent
sqlCtx.registerDataFrameAsTable(temp_reviews, "temp_reviews")
temp_reviews=sqlCtx.sql('''SELECT *\
, get_sentiment(text,title) AS sentiment\
, get_recentReviews(date) AS recent_reviews\
, 1 AS nb_reviews \
FROM temp_reviews''')

# Countries
sqlCtx.registerDataFrameAsTable(temp_reviews, "temp_reviews")
list_countries = [ x.country.replace(" ","_") for x in sqlCtx.sql("SELECT country \
 FROM temp_reviews \
 group by country \
 ").collect()]

# Countries expanded
for c in list_countries:
    sqlCtx.registerDataFrameAsTable(temp_reviews, "temp_reviews")
    temp_reviews=sqlCtx.sql('''SELECT *\
    , CASE WHEN (escape(country) = "'''+c+'''") THEN 1 \
    ELSE 0 END AS '''+c.replace("-","_")+'''\
    FROM temp_reviews''')
sqlCtx.registerDataFrameAsTable(temp_reviews, "temp_reviews")


In [ ]:
#Group the reviews first step (need to modify both in case a new column is added)
fields_to_group = ["nb_reviews","recent_reviews"]
for l in list_languages:
    fields_to_group.append(l)
for c in list_countries:
    fields_to_group.append(c)
    
sql_quiery = "SELECT id, AVG(sentiment) AS sentiment, COUNT(DISTINCT version) AS versions"
for f in fields_to_group:
    sql_quiery+= ", SUM("+f+") AS "+f
sql_quiery+= " FROM temp_reviews GROUP BY id"

#print sql_quiery

grp_reviews = sqlCtx.sql(sql_quiery)


In [ ]:
#Group the reviews second step (need to modify both in case a new column is added)
#Make the Gini and other post grouping predictors
def get_gini(*args):
    args = list(args)
    tot = sum(args)
    return sum([1.0*x / tot * (1 - 1.0*x / tot) for x in args])
sqlCtx.registerFunction("get_gini", get_gini,returnType=FloatType())

sqlCtx.registerDataFrameAsTable(grp_reviews, "grp_reviews")
sql_quiery = "SELECT id , sentiment, nb_reviews,recent_reviews,versions"
for c in list_countries:
    sql_quiery+= ","+c
sql_quiery+= " , get_gini(0"
for l in list_languages:
    sql_quiery+= ","+l
sql_quiery+= ") AS gini_reviews FROM grp_reviews"
    
grp_reviews = sqlCtx.sql(sql_quiery)

predictors = predictors.join(grp_reviews,["id"],"left")

In [ ]:
predictors.head()

In [ ]:
reviews = reviews.head(100)
reviews = sqlCtx.createDataFrame(reviews)

In [ ]:
grp_reviews.head()


In [ ]:
get_std([1,2,3,4])

In [ ]:
Row(**dict(row.asDict()))

In [ ]:
(lambda row: Row(**dict(row.asDict(), test=-1)))(row)

In [ ]:
downloads.head(1)

In [ ]:
grp_reviews.head(1)